<a href="https://colab.research.google.com/github/LeDucDiLac/anti-spoofing-w-dinov2/blob/main/Dinov2_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision.transforms import v2
import cv2
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using", device)

dino_name = "dinov2_vitg14"
dino_model = torch.hub.load("facebookresearch/dinov2", dino_name)
dino_model.to(device).eval()

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
from torch import nn

class LargerNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LargerNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(512, 256)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

if(dino_name == "dinov2_vitg14"):
  input_dim = 1536
elif dino_name == "dinov2_vitl14":
  input_dim = 1024
elif dino_name == "dinov2_vitb14":
  input_dim = 768
elif dino_name == "dinov2_vits14":
  input_dim = 384

output_dim = 2
nn_model_path = "/content/drive/MyDrive/antispoofing/modelg14_4l.pth"
nn_model = LargerNN(input_dim, output_dim)
nn_model.load_state_dict(torch.load(nn_model_path, weights_only=True, map_location=device))
nn_model.to(device).eval()


img_size = 224
transform = v2.Compose([
    v2.Resize((img_size,img_size)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
nn_model.eval()
all_labels = []
all_predictions = []
misclassified_paths = []

with torch.no_grad():
    correct = 0
    total = 0
    for batch_features, batch_labels, batch_paths in test_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        outputs = nn_model(batch_features)
        _, predictions = torch.max(outputs, 1)

        all_labels.extend(batch_labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        total += batch_labels.size(0)
        correct += (predictions == batch_labels).sum().item()

        # Identify misclassified images
        for i, (label, pred, path) in enumerate(zip(batch_labels.cpu().numpy(), predictions.cpu().numpy(), batch_paths)):
            if label != pred:
                misclassified_paths.append((path, label, pred))

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
# Confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=torch.unique(labels).numpy())
disp.plot(cmap='viridis', values_format='d')
plt.title('Confusion Matrix')
plt.show()
